# NB #2:  Arbitrage 

### Pool address: 0xFA2562da1Bba7B954f26C74725dF51fb62646313  -- AAVE 30%; SNX 25%; SUSHI 15%; YFI 30%

The Balancer Pool cadCAD model provides a core infrastructure for simulating Balancer Pools in the **Token Engineering Design and Verification process**.

It allows to simulate Balancer Pool transactions and verify new designs and application cases.  
In this notebook we plug **historical on-chain data** to a cadCAD model, and analyse **Pool Power** and **Pool Characteristics**.

For more information check out the [Balancer Simulations documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/).

# A. System Context

### A1.1 System Specification
System specification details are available in the Balancer Simulations documentation:
- [Differential Specification](https://token-engineering-balancer.gitbook.io/balancer-simulations/balancer-simulations/v10nboverview)
- [Mathematical Specification](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/balancer-the-python-edition/balancer_math.py)
- [Model Architecture](https://token-engineering-balancer.gitbook.io/balancer-simulations/balancer-simulations/v10nboverview)
- [Naming Convention](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/naming-convention)

# B. cadCAD Simulations

### B1.1 Dependencies

In [ ]:
import pandas as pd 
from cadCAD.configuration.utils import config_sim
from model.models import Token, Pool
from model.parts.pool_method_entities import TokenAmount

### B1.2 Initialize Pool


In this section you specify the Balancer Pool to be analyzed and run scripts to produce the'genesis_state', based on on-chain data. 

More information here: [Balancer Simulations documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/onchaintransactions).  

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

from decimal import Decimal

# No spot_price_reference means all pairs are calculated.
parameters = {
    'decoding_type': ['SIMPLIFIED'],
    'external_currency': ['USDT'],
    'max_arb_liquidity': [100000],
    'min_arb_liquidity': [10],
    'arb_liquidity_granularity': [50],
    'weight_changing': [False]
    
}


#### Create your pool

Create your genesis states using our Token and Pool classes. Pool will check for token weight integrity

Token prices could come from initial state files

// TODO document
Learn more how to create this variables [documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/v/master/todo).


In [ ]:
from model.genesis_states import generate_initial_state

initial_values = generate_initial_state("data/0xfa2562da1bba7b954f26c74725df51fb62646313-initial_pool_states-prices.json", "USD")

print(initial_values)


#### State Variables and Initial Values

In [ ]:
print('## State Variables')
print('# Pool')
initial_values['tx_cost'] = TokenAmount(symbol='USD', amount=Decimal(30))
initial_values['change_datetime'] = '2020-12-07T13:34:14+00:00'
initial_values['action_type'] = 'pool_creation'

pp.pprint(initial_values)


#### External USD Token Prices

In [ ]:
print('# External token prices, initial state')
token_prices = initial_values['token_prices']
pp.pprint(token_prices)

print('# Action Type')
action_type = initial_values['action_type']
pp.pprint(action_type)


### B1.3 State Update Functions & Policies

Balancer Simulations replicate Balancer Pool Transactions in state update functions.  
For a detailed description, please visit the [Balancer Simulations documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/balancer-the-python-edition).

### B1.4 Partial State Update Blocks

Partial State Update Blocks combine the following steps:  
1. **Initialize pool**, generate genesis state
2. Compute subsequent **actions in discrete timesteps**, store datetime and update the pool state variables
3. Update external **USD prices in discrete timesteps**, and store datetime  

For more information please visit the [Balancer Simulations documentation/Model Overview](https://token-engineering-balancer.gitbook.io/balancer-simulations/v/master/balancer-simulations/v10nboverview).  

To inject historical on-chain transactions to the model, reference the **actions.json of your pool below**.

In [ ]:
from model.partial_state_update_block import generate_partial_state_update_blocks

result = generate_partial_state_update_blocks(path_to_action_json='data/0xfa2562da1bba7b954f26c74725df51fb62646313-actions.json',
                                              add_arbitrage=False)
partial_state_update_blocks = result['partial_state_update_blocks']
pp.pprint(partial_state_update_blocks)

### B1.5 Configuration

[cadCAD simulations](https://github.com/cadCAD-org/cadCAD/blob/master/documentation/README.md) support Monte Carlo runs, and parameter sweeps which are not applied in this notebook and can be ignored here.  
Timesteps 'T' are defined by your pool's .json file, no need to make edits here.

In [ ]:
steps_number = result['steps_number']
print('# Steps ', steps_number)
sim_config = config_sim(
    {
        'N': 1,  # number of monte carlo runs
        'T': range(steps_number - 1),  # number of timesteps
        'M': parameters,  # simulation parameters
    }
)

### B1.6 Execution

In [ ]:
from model.sim_runner import *
from ipdb import launch_ipdb_on_exception


with launch_ipdb_on_exception():
    df = run(initial_values, partial_state_update_blocks, sim_config)

### B1.7 Simulation Output Preparation

Post-processing (utils.py) splits up state variable dictionaries, and adds metrics to the data frame, such as 
- `token_total_value` (TVL)
- `generated_fees_(tokensymbol)` (fee collected per transaction)
- `token_k_values`  (token value in USD)  
based on  
- `token_k_balances`
- `token_k_price`  (external price feed)

In [ ]:
from data.utils import save_pickle, load_pickle
# save_pickle(df, "NB3df.pickle")
# df = load_pickle("NB3-noswaps.pickle")

In [ ]:
from model.parts.utils import post_processing
p_df = post_processing(df, include_spot_prices=True)
# import ipdb; ipdb.set_trace()
# p_df = p_df[p_df['substep'] == 2]
p_df.tail()

# C. Simulation Outcome & Pool Exploration

The plots below offer some keys metrics for pool analysis.

**a) Pool Power:**  
- C1.1 TVL (Total Value Locked) vs. Pool Growth (token balances in the pool, over time)
- C1.2 Token Balances (individual token balances, over time)

**b) Pool Characteristics:**  
- C1.3 Sources of Pool Growth (BPT, Fees)
- C1.4 Token Ratio
- C1.5 Action Types

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
print("Observation Time")
print(f"Start: {p_df['change_datetime'].min()}")
print(f"End: {p_df['change_datetime'].max()}")
print(f"Total Observation Period: {p_df['change_datetime'].max() - p_df['change_datetime'].min()}")
print("\n")
print(f"Total No. of Timesteps (incl. Price Updates): {(p_df.iloc[-1]['timestep'])}")
print(f"Total No. of Transactions: {len(p_df[p_df.action_type != 'external_price_update'])}")

### Balances

In [ ]:
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_snx_balance, line=dict(color='green'), name="Balance SNX"), secondary_y=False)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_sushi_balance, line=dict(color='blue'), name="Balance SUSHI"), secondary_y=False)

fig.update_layout(title_text='<b>Token Balances</b>')
fig.update_xaxes(title_text='timestep')

fig.show()  

In [ ]:
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_aave_balance, line=dict(color='red'), name="Balance AAVE"), secondary_y=False)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_yfi_balance, line=dict(color='black'), name="Balance YFI"), secondary_y=True)

fig.update_layout(title_text='<b>Token Balances</b>')
fig.update_xaxes(title_text='timestep')

fig.show()  

In [ ]:
### Prices

In [ ]:
# fig = make_subplots(specs=[[{'secondary_y': True}]])
fig = make_subplots(rows=3, cols=2, subplot_titles=('External Price AAVE', 'External Price SNX', 'External Price SUSHI', 'External Price YFI', 'Spot Price AAVE YFI'))

fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_aave_price, line=dict(color='red'), name="External Price AAVE"), secondary_y=False, row=1, col=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_snx_price, line=dict(color='green'), name="External Price SNX"), secondary_y=False, row=1, col=2)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_sushi_price, line=dict(color='blue'), name="External Price SUSHI"), secondary_y=False, row=2, col=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_yfi_price, line=dict(color='black'), name="External Price YFI"), secondary_y=False, row=2, col=2)

fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_spot_price_aave_yfi, line=dict(color='#803050'), name="Spot Price AAVE YFI"), row=3, col=1)

fig.update_layout(title_text='<b>External Prices</b>')
fig.update_xaxes(title_text='timestep')

fig.show()  

### C1.1 TVL & Pool Size Growth

In [ ]:
#TVL vs. Pool Size Growth
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.tvl, line=dict(color='#f36315'), name="TVL total_token_value"), secondary_y=False)
fig.add_trace(go.Scatter(x=p_df.timestep,y=p_df.total_token_balances, line=dict(color='#2C1839'), name="total_token_balance"), secondary_y=True)
fig.update_layout(title_text='<b>TVL vs. Pool Growth</b>')
fig.update_xaxes(title_text='timestep')
fig.update_yaxes(title_text='<b>TVL total_token_value</b> in USD', secondary_y=False)
fig.update_yaxes(title_text='<b>total_token_balance</b> in #', secondary_y=True)
fig.show()  

### C1.1 Pool Invariant & Pool Size Growth

In [ ]:
#TVL vs. Pool Size Growth
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.invariant, line=dict(color='red'), name="Invariant in #"), secondary_y=False)
fig.add_trace(go.Scatter(x=p_df.timestep,y=p_df.total_token_balances, line=dict(color='#2C1839'), name="total_token_balance"), secondary_y=True)
fig.update_layout(title_text='<b>Invariant vs. Pool Growth</b>')
fig.update_xaxes(title_text='timestep')
fig.update_yaxes(title_text='<b>Invariant</b> in #', secondary_y=False)
fig.update_yaxes(title_text='<b>total_token_balance</b> in #', secondary_y=True)
fig.show()  

### C1.2 Token Balances

In [ ]:
#balance growth % (total observation period)
aave_growth = ((p_df.iloc[-1][['token_aave_balance']])-(p_df.iloc[0][['token_aave_balance']]))*100/(p_df.iloc[0][['token_aave_balance']])
snx_growth = ((p_df.iloc[-1][['token_snx_balance']])-(p_df.iloc[0][['token_snx_balance']]))*100/(p_df.iloc[0][['token_snx_balance']])
sushi_growth = ((p_df.iloc[-1][['token_sushi_balance']])-(p_df.iloc[0][['token_sushi_balance']]))*100/(p_df.iloc[0][['token_sushi_balance']])
yfi_growth = ((p_df.iloc[-1][['token_yfi_balance']])-(p_df.iloc[0][['token_yfi_balance']]))*100/(p_df.iloc[0][['token_yfi_balance']])
growth = list(zip(aave_growth, snx_growth, sushi_growth, yfi_growth)) 
g_df = pd.DataFrame(growth, columns = ['aave_growth','snx_growth','sushi_growth','yfi_growth' ]).transpose(copy=True).reset_index()
g_df.columns =['token', 'growth']
print(g_df)
#plot
fig = make_subplots(rows=3, cols=2, subplot_titles=('AAVE Growth', 'Balance Growth in %', 'SNX Growth', 'SUSHI Growth', 'YFI Growth'))
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_aave_balance, line=dict(color='#4675ed'), name='token_aave_balance'), row=1, col=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_snx_balance, line=dict(color='#4145ab'), name='token_snx_balance'), row=2, col=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_sushi_balance, line=dict(color='#3135ab'), name='token_sushi_balance'), row=2, col=2)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_yfi_balance, line=dict(color='#2125ab'), name='token_yfi_balance'), row=3, col=1)
fig.add_trace(go.Bar(y=g_df.growth, x=g_df.token, marker_color=['#4675ed','#4145ab']), row=1, col=2)
fig.update_layout(height=600, width=1000, showlegend=False, title_text='<b>Balance Growth</b> in observation period')
fig.show()

### C1.3 Sources of Growth 

In [ ]:
#Shares vs Share Price Growth
fig = make_subplots(specs=[[{'secondary_y': True}]])
initial_share_price = p_df.share_price.iloc[0]
share_price_normalized = p_df.share_price / initial_share_price
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.shares, line=dict(color='red'), name="Shares"), secondary_y=False)
fig.add_trace(go.Scatter(x=p_df.timestep,y=share_price_normalized, line=dict(color='#2C1839'), name="Share Price Growth"), secondary_y=True)
fig.update_layout(title_text='<b>Shares (BPT) vs Share Price Growth</b>')
fig.update_xaxes(title_text='timestep')
fig.update_yaxes(title_text='<b>Shares</b> in # Tokens', secondary_y=False)
fig.update_yaxes(title_text='<b>Share Price Growth</b> (normalized)', secondary_y=True)
fig.show()  

In [ ]:
#Fees
fig = make_subplots(rows=4, cols=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.generated_fees_aave, line=dict(color='red'), name='fees AAVE'), row=1, col=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.generated_fees_snx, line=dict(color='green'), name='fees SNX'), row=2, col=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.generated_fees_sushi, line=dict(color='blue'), name='fees SUSHI'), row=3, col=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.generated_fees_yfi, line=dict(color='black'), name='fees YFI'), row=4, col=1)
fig.update_yaxes(title_text='token amount', secondary_y=False)
fig.update_layout(height=500, width=1000, title_text='<b>Fees Generated</b> per transaction')

In [ ]:
# DAI fee events vs. ETH fee events
AAVE_fee_events = len(p_df[p_df.generated_fees_aave > 0])
SNX_fee_events = len(p_df[p_df.generated_fees_snx > 0])
SUSHI_fee_events = len(p_df[p_df.generated_fees_sushi > 0])
YFI_fee_events = len(p_df[p_df.generated_fees_yfi > 0])

values = [AAVE_fee_events, SNX_fee_events, SUSHI_fee_events, YFI_fee_events]
labels = ['AAVE fees', 'SNX fees', 'SUSHI fees', 'YFI fees']
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(hoverinfo='label+percent', marker=dict(colors=['red','green','blue','black']))
fig.update_layout(height=500, width=1000, title_text='<b>Fee generating events</b> in total observation time')
fig.show()

In [ ]:
#proportions of fee value contribution, in fixed USD value
aavef = p_df.groupby('action_type').sum()['generated_fees_aave'].drop(['external_price_update'])
snxf = p_df.groupby('action_type').sum()['generated_fees_snx'].drop(['external_price_update'])
sushif = p_df.groupby('action_type').sum()['generated_fees_sushi'].drop(['external_price_update'])
yfif = p_df.groupby('action_type').sum()['generated_fees_yfi'].drop(['external_price_update'])
fee_v = pd.concat([aavef, snxf, sushif, yfif], axis=1)

#calculate USD value (define USD value, in this case locked to initial state observation time)
#calculate USD value (define USD value, in this case locked to initial state observation time)
fee_v['AAVE_fee_value'] = fee_v.generated_fees_aave*489.4165 #define AAVE price
fee_v['SNX_fee_value'] = fee_v.generated_fees_snx*21.305500000000002 #define SNX price
fee_v['SUSHI_fee_value'] = fee_v.generated_fees_sushi*14.125 #define SUSHI price
fee_v['YFI_fee_value'] = fee_v.generated_fees_yfi*32569.655 #define YFI price

#add total 
fee_v = fee_v.append(fee_v.sum().rename('total'))
print(fee_v)

#plot pie chart comparing ETH fees vs DAI fees
values = [fee_v.loc['total']['AAVE_fee_value'], fee_v.loc['total']['SNX_fee_value'], fee_v.loc['total']['SUSHI_fee_value'], fee_v.loc['total']['YFI_fee_value']] 
labels = ['AAVE_fees', 'SNX_fees', 'SUSHI_fees', 'YFI_fees']
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(hoverinfo='label+percent', marker=dict(colors=['red','green', 'blue', 'black']))
fig.update_layout(height=500, width=1000, title_text="<b>Fee value contribution</b> in USD, in total observation time")
fig.show()

In [ ]:
# FEE PER DAY/in fixed USD value

#calculate USD value (define USD value, in this case locked to initial state observation time)
fee_t = p_df[['change_datetime']].dropna()
fee_t['AAVE_fee_value'] = p_df.generated_fees_aave*489.4165 #define AAVE price
fee_t['SNX_fee_value'] = p_df.generated_fees_snx*21.305500000000002 #define SNX price
fee_t['SUSHI_fee_value'] = p_df.generated_fees_sushi*14.125 #define SUSHI price
fee_t['YFI_fee_value'] = p_df.generated_fees_yfi*32569.665 #define YFI price
daily = fee_t.groupby(pd.Grouper(key='change_datetime',freq='D')).sum() #sum per week

#plot value per week
fig = go.Figure(data=[
	go.Bar(name='AAVE fees', x=daily.index, y=daily.AAVE_fee_value, marker_color='red', offsetgroup=0),
	go.Bar(name='SNX fees', x=daily.index, y=daily.SNX_fee_value, marker_color='green', offsetgroup=1),
	go.Bar(name='SUSHI fees', x=daily.index, y=daily.SUSHI_fee_value, marker_color='blue', offsetgroup=2),
	go.Bar(name='YFI fees', x=daily.index, y=daily.YFI_fee_value, marker_color='black', offsetgroup=3)], layout=go.Layout(title='<b>Fees per day</b> in USD', yaxis_title='USD value'))
fig.show()

### C1.4 Token Ratio

In [ ]:
events = [0, -1] #Start/End of observation period
ratio = pd.DataFrame(p_df.iloc[events][['token_aave_balance','token_snx_balance','token_sushi_balance','token_yfi_balance']]).transpose(copy=True)
ratio.columns =['start', 'end'] 
#plot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=ratio.index, values=ratio.start), 1, 1)
fig.add_trace(go.Pie(labels=ratio.index, values=ratio.end), 1, 2)
fig.update_traces(hoverinfo='label+percent+name', marker=dict(colors=['#4675ed','#4145ab']))
fig.update_layout(height=400, width=1000, title_text='<b>Token Ratio</b> Start/End Observation Period in %')
fig.show()